In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np

In [0]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np

class PRC_RCD_PROVISIONES():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def clasificacionDeudorPorcentaje(self,fechaProceso):
    
    select_clasificacion_deudor_vigente="""SELECT CLASIFICACION_DEUDOR, TIPO_CREDITO_SBS, CONCEPTO_SALDO, PORCENTAJE/100 AS PORCENTAJE, TABLA, FECHA_VIGENCIA
    FROM CLASIFICACION_DEUDOR_PORCENTAJE
    WHERE """"'"+ fechaProceso +"""' >= FECHA_VIGENCIA
    AND ACTIVO=1
    """
    sql_Clasif_Deudor_vigente = pd.read_sql_query(select_clasificacion_deudor_vigente, self.connection)
    df_Clasif_Deudor_vigente = pd.DataFrame(sql_Clasif_Deudor_vigente)
    df_Clasif_Deudor_vigente.sort_values(by=['FECHA_VIGENCIA'], inplace=True, ascending=False)
      
    df_Clasif_Deudor_vigente['RANK'] = df_Clasif_Deudor_vigente.groupby(by=['CLASIFICACION_DEUDOR','TIPO_CREDITO_SBS','CONCEPTO_SALDO','TABLA'])['FECHA_VIGENCIA'].transform(lambda x: x.rank())
    filtro_rank = df_Clasif_Deudor_vigente["RANK"]==1
    df_Clasif_Deudor_vigente.where(filtro_rank, inplace = True)
    
    return df_Clasif_Deudor_vigente
  
  def cuentaContableVigente(self,fechaProceso):
    
    select_cuenta_contable_vigente="""SELECT ID,
    EMPRESA,
    PRODUCTO,
    TIPO_CLIENTE,
    TIPO_CREDITO,
    TIPO_CREDITO_SBS,
    SITUACION_CREDITO,
    INDICADOR_PAGO_CASTIGO,
    CLASIFICACION_DEUDOR,
    CONCEPTO_SALDO_PRINCIPAL,
    CONCEPTO_SALDO,
    CONCEPTO_CLASIFICACION,
    MONEDA,
    INFORME_SBS,
    CONDICION_ESPECIAL,
    CONDICION_DISPONIBILIDAD,
    FACTOR_CONVERSION_CREDITO,
    CONDICION_DIAS,
    CUENTA_CONTABLE,
    FECHA_VIGENCIA FROM CUENTA_CONTABLE
    WHERE """"'"+ fechaProceso +"""' >= FECHA_VIGENCIA
    """
    sql_cuenta_contable_vigente = pd.read_sql_query(select_cuenta_contable_vigente, self.connection)
    df_cuenta_contable_vigente = pd.DataFrame(sql_cuenta_contable_vigente)
    
    #ordernar por fechaVigengia de forma descendente
    df_cuenta_contable_vigente.sort_values(by=['FECHA_VIGENCIA'], inplace=True, ascending=False)
    
    df_cuenta_contable_vigente['RANK'] = df_cuenta_contable_vigente.groupby(by=['EMPRESA','PRODUCTO','TIPO_CLIENTE','TIPO_CREDITO','TIPO_CREDITO_SBS','INDICADOR_PAGO_CASTIGO','CLASIFICACION_DEUDOR','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_SALDO','CONCEPTO_CLASIFICACION','MONEDA'])['FECHA_VIGENCIA'].transform(lambda x: x.rank())
    filtro_rank = df_cuenta_contable_vigente["RANK"]==1
    #df_cuenta_contable_vigente.where(filtro_rank, inplace = True)
    
    return df_cuenta_contable_vigente
      
  def CalculoDeProvisiones(self):
    
    #Reproceso
    eliminaProvision = """DELETE  CPC
    FROM    CLIENTE C
            JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
            JOIN CLIENTE_PRODUCTO_CONCEPTO CPC ON CPC.CLIENTE_PRODUCTO = CP.ID
    WHERE   FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    AND CPC.CONCEPTO_SALDO = 20;
    """
    self.cursor.execute(eliminaProvision)
    
    eliminaProvisionProciclica = """DELETE  CPC
    FROM    CLIENTE C
            JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
            JOIN CLIENTE_PRODUCTO_CONCEPTO CPC ON CPC.CLIENTE_PRODUCTO = CP.ID
    WHERE   FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    AND CPC.CONCEPTO_SALDO = 80;
    """
    self.cursor.execute(eliminaProvisionProciclica)
    
    eliminaProvisionVoluntaria="""DELETE  CPC
    FROM    CLIENTE C
            JOIN CLIENTE_PRODUCTO CP ON C.ID = CP.CLIENTE
            JOIN CLIENTE_PRODUCTO_CONCEPTO CPC ON CPC.CLIENTE_PRODUCTO = CP.ID
    WHERE   FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    AND CPC.CONCEPTO_SALDO = 111;
    """
    self.cursor.execute(eliminaProvisionVoluntaria)
    self.connection.commit()
    
    productoConcepto ="""SELECT B.ID CLIENTE_PRODUCTO,
                       E.EMPRESA,
                       D.PRODUCTO,
                       B.TIPO_CLIENTE,
                       B.TIPO_CREDITO,
                       B.TIPO_CREDITO_SBS,
                       B.SITUACION_CREDITO,
                       B.INDICADOR_PAGO_CASTIGO,
                       A.CLASIFICACION_DEUDOR,
                       C.CONCEPTO_SALDO_PRINCIPAL,
                       C.CONCEPTO_SALDO,
                       C.CONCEPTO_CLASIFICACION,
                       C.MONEDA,
                       CASE WHEN ISNULL(G.OPERADOR,'+') = '-' THEN
                          CASE WHEN C.CONCEPTO_SALDO_PRINCIPAL = 83 THEN C.IMPORTE*1.0 *-1 ELSE C.IMPORTE*-1 END
                       ELSE
                          CASE WHEN C.CONCEPTO_SALDO_PRINCIPAL = 83 THEN C.IMPORTE*1.0     ELSE C.IMPORTE END
                       END IMPORTE,
                       CASE WHEN ISNULL(G.OPERADOR,'+') = '-' THEN
                          CASE WHEN C.CONCEPTO_SALDO_PRINCIPAL = 83 THEN C.IMPORTE_INGRESO*1.0 *-1 ELSE C.IMPORTE_INGRESO*-1 END
                       ELSE
                          CASE WHEN C.CONCEPTO_SALDO_PRINCIPAL = 83 THEN C.IMPORTE_INGRESO*1.0     ELSE C.IMPORTE_INGRESO END
                       END IMPORTE_INGRESO
    FROM CLIENTE A
    JOIN CLIENTE_PRODUCTO B
    ON A.ID = B.CLIENTE
    JOIN CLIENTE_PRODUCTO_CONCEPTO C
    ON B.ID = C.CLIENTE_PRODUCTO
    JOIN CLIENTE_DEUDOR_PRODUCTO D
    ON D.ID = B.CLIENTE_DEUDOR_PRODUCTO
    JOIN CLIENTE_DEUDOR E
    ON E.ID = D.CLIENTE_DEUDOR
    JOIN PROCESO_PRODUCTO F
    ON F.PROCESO = 4 --Provisiones
    AND F.PRODUCTO = D.PRODUCTO
    JOIN PROCESO_PRODUCTO_DETALLE G
    ON F.ID = G.PROCESO_PRODUCTO
    AND G.GRUPO_CONCEPTO_SALDO = 1
    AND G.CONCEPTO_SALDO = C.CONCEPTO_SALDO
    AND G.SITUACION_CREDITO = B.SITUACION_CREDITO
    WHERE A.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    """
    sql_productoConcepto = pd.read_sql_query(productoConcepto, self.connection)
    df_productoConcepto = pd.DataFrame(sql_productoConcepto)
        
    select1 = pd.merge(df_productoConcepto, PRC_RCD_PROVISIONES.clasificacionDeudorPorcentaje(self.fecha_proceso_str),on='CLASIFICACION_DEUDOR', how='left')

    select1['TIPO_CREDITO_SBS_x'] = np.where(select1['TABLA'] > 0, select1['TIPO_CREDITO_SBS_y'], select1['TIPO_CREDITO_SBS_x'])
    
    select1['TABLA'] = np.where(select1['CLASIFICACION_DEUDOR'] > '0', 1, np.where(select1['CLASIFICACION_DEUDOR'] == 0, 0, -1))
    rfselect = pd.DataFrame(select1)
    
    select1.drop(['TIPO_CREDITO_SBS_y','CONCEPTO_SALDO_x'], axis="columns", inplace=True) 
    select1.rename(columns={'TIPO_CREDITO_SBS_x':'TIPO_CREDITO_SBS', 'CONCEPTO_SALDO_y':'CONCEPTO_SALDO'}, inplace=True)
    rfselect1 = pd.DataFrame(select1)
    
    select2 = pd.merge(rfselect1, PRC_RCD_PROVISIONES.cuentaContableVigente(self.fecha_proceso_str),on=['EMPRESA','PRODUCTO','TIPO_CLIENTE','TIPO_CREDITO','TIPO_CREDITO_SBS','SITUACION_CREDITO','INDICADOR_PAGO_CASTIGO','CLASIFICACION_DEUDOR','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_SALDO','CONCEPTO_CLASIFICACION','MONEDA'], how='left')
    
    #filtramos por los campos que necesitamos
    select2.filter(['CLIENTE_PRODUCTO','CONCEPTO_SALDO','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_CLASIFICACION','MONEDA','ID','IMPORTE','IMPORTE_INGRESO','PORCENTAJE'])
    
    #calculamos la provision según el porcentaje del deudor
    select2['IMPORTE'] = select2['IMPORTE']*select2['PORCENTAJE']
    select2['IMPORTE_INGRESO'] = select2['IMPORTE_INGRESO']*select2['PORCENTAJE']
    
    #agrupamos
    select2.groupby(by =['CLIENTE_PRODUCTO','CONCEPTO_SALDO_PRINCIPAL','CONCEPTO_SALDO','CONCEPTO_CLASIFICACION','MONEDA','ID'])(['IMPORTE'],['IMPORTE_INGRESO']).sum()
    
    #insertamos las provisiones
    for index,row in select2.iterrows():
      self.cursor.execute('INSERT INTO dbo.CLIENTE_PRODUCTO_CONCEPTO([CLIENTE_PRODUCTO],[CONCEPTO_SALDO_PRINCIPAL],[CONCEPTO_SALDO],[CONCEPTO_CLASIFICACION],[MONEDA],[IMPORTE],[IMPORTE_INGRESO]) values (?,?,?,?,?,?,?)', 
                      row['CLIENTE_PRODUCTO'], 
                      row['CONCEPTO_SALDO_PRINCIPAL'], 
                      row['CONCEPTO_SALDO'],
                      row['CONCEPTO_CLASIFICACION'],
                      row['MONEDA'],
                      row['IMPORTE'],
                      row['IMPORTE_INGRESO'])
      self.connection.commit()
      self.cursor.close()
      
#PRC_RCD_PROVISIONES = PRC_RCD_PROVISIONES(1)
#PRC_RCD_PROVISIONES.cuentaContableVigente('2021-03-31')